In [2]:
######################################
## Test data for debugging purposes ##
######################################

# import random
# random.seed(7)

# total_promocodes = 25
# total_orders = 10000

# promocodes = {
#     "promocode_id": list(map(str, range(0, total_promocodes))), 
#     "name": list(string.ascii_lowercase)[0:total_promocodes],
#     "discount": np.random.choice(np.linspace(10, 1000, num = 100), total_promocodes, replace = False)
# }

# orders = {
#     "order_id": list(range(0, total_orders)),
#     "promocode_id": np.random.choice(np.append(promocodes["promocode_id"], "NULL"), total_orders, replace = True)
# }

# df_promo = pd.DataFrame(data = promocodes)
# df_orders = pd.DataFrame(data = orders)

## Задача 1
Сервис Яндекс.Еда привлекает новых клиентов в том числе с помощью скидок. При этом для повышения экономической эффективности необходимо отслеживать основные метрики по скидкам. Напишите код, который отвечает на следующие вопросы (можно использовать SQL или Python)
- Какая доля заказов с промокодами?
- Самый популярный (в метрике суммы скидки) промокод (название) и сумма скидки по нему?

Данные, которые есть (для простоты в формате .txt, несколько первых строк): <br>
<b>orders.txt</b> (если в заказе был промокод, то записываеся id промокода; если не было, то NULL) <br>
<b>promocodes.txt</b> (discount – сумма скидки в рублях)

In [1]:
## Initialization of libraries & data importing
import numpy as np
import pandas as pd
import string

df_orders = pd.read_csv("orders.txt", sep = ",")
df_promo = pd.read_csv("promocodes.txt", sep = ",")

In [4]:
## Creating general Data Frame with promo codes and orders information via left join
df_total = pd.merge(df_orders, df_promo, on = "promocode_id", how = "left")

## Cutting of NULL promocodes since we don't need them
null_share = df_total[df_total["promocode_id"] == "NULL"].agg("count")["order_id"]

print(
    "Share of orders with promo codes:", 
    str((1 - null_share/df_total.shape[0])*100) + "%"
)

## I believe there could be an optimization in counting ONLY NULL orders 
## But this would work only if total share of people using promo codes dominates over those who don't

Share of orders with promo codes: 96.17%


In [6]:
## Aggregating df_total with sum of discount and left-joining promo codes to acquire name of the most efficient code
discounts = pd.merge(
    df_total.groupby("promocode_id")["discount"].agg("sum"), 
    df_promo.drop("discount", axis = 1), 
    on = "promocode_id", how = "left"
)

print(
    "The most popular promo code (by total saves) is:",
    discounts.loc[discounts["discount"].idxmax()]["name"],
    "\nWith", int(discounts["discount"].max()), "total saves"
)

## There is probably an easier way to manipulate data since we DO have promo code names in df_total
## But, unfortunatelly pandas.DataFrame.agg documentations is showing only basic functionality
## Not the best example of using python/pandas data structures, but I tried ¯\_(ツ)_/¯

The most popular promo code (by total saves) is: l 
With 384560 total saves


## Задача 2
Яндекс.Еда осуществляет доставку еды из ресторанов. При этом у каждого ресторана есть зона, в рамках которой осуществляется доставка. Зона представляет собой полигон из координат. Пользователь в зависимости от своей координаты видит разное количество доступных ресторанов. Нам важно, чтобы у каждого пользователя было достаточное количество ресторанов для выбора. Задача заключается в том, чтобы по каждой координате пользователя посчитать доступное ему количество ресторанов. Можно использовать SQL или Python.

Данные, которые есть (для простоты в формате .txt, несколько первых строк): <br>
<b>user_coordinates.txt</b> (примерно 300 тыс строк, user_id – идентификатор пользователя) <br>
<b>place_zone_coordinates</b> (примерно 500 тыс строк, place_id – идентификатор ресторана, point_number – порядковый номер точки в зоне доставки ресторана) 

## Задача 3
Мы в Яндекс.Еде любим поиграть в покер и настольные игры. Поэтому нам важно, чтобы будущие коллеги были сильны в математической статистике и теории вероятностей (так интереснее играть))) Если серьезно, то в большинстве аналитических задач у нас возникает необходимость применять математический аппарат в том или ином виде. Проще всего его проверить через учебные игровые задачи, чтобы не погружать в сложный контекст бизнеса.
Вы подбрасываете кубик (6 граней), после чего у вас есть две возможности:
1. Взять себе сумму \\$, равную выпавшему на кубике числу (выпало 3 – получаете 3\\$)
2. Отклонить результат первого броска и подбросить кубик второй раз. После чего уже взять себе столько \\$, сколько выпало во второй раз (второй бросок отклонять уже нельзя)
Какую стратегию нужно выбрать, чтобы максимизировать ожидаемую прибыль (при каких значениях первого броска нужно перебрасывать?)<br> 
Почему эта стратегия максимизирует прибыль? <br>
Чему равна ожидаемая прибыль при этой стратегии (**математическое ожидание**)?
